# Prácticas ESAC: Strömgren Radius Tilvi20

1. Extracted data from Tilvi20
2. Filter in redshift
3. Strömgen Radius

   Appendix A: Saving data

In [8]:
# Paquetes
from astropy.table import Table, QTable, unique
from astropy.io import ascii
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import smplotlib
from uncertainties import ufloat, unumpy
import uncertainties.unumpy as unp
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import sys

# Definimos Mpc en unidades propias comóviles
pMpc = u.def_unit('pMpc')
cMpc = u.def_unit('cMpc')

## 1. Extracted data from Tilvi20

A continuación, importamos la _Table 1_ del paper 11 (V. Tilvi et al. (2020)).

In [2]:
# Creamos la tabla de datos de Tilvi20
data_Tilvi20 = {
    'ID': ['z8_SM', 'z8_4', 'z8_5'],
    'RA_J2000': ['14:20:35.694', '14:20:35.169', '14:20:34.872'],
    'DEC_J2000': ['+53:00:09.318', '+52:59:40.613', '+53:00:15.242'],
    'NB': [24.76, 23.85, 23.60]*u.mag,
    'e_NB': [0.35, 0.15, 0.12]*u.mag,
    'F125W': [26.76, 27.25, 25.29]*u.mag,
    'e_F125W': [0.13, 0.21, 0.03]*u.mag,
    'F160W': [26.66, 27.10, 25.08]*u.mag,
    'e_F160W': [0.11, 0.16, 0.03]*u.mag,
    'z_spec': [7.767, 7.748, 7.728],
    'logfLya': np.round(np.log10([0.29*10**(-17), 0.56*10**(-17), 1.7*10**(-17)]),3)*u.dex(u.erg/u.s),
    'e_logfLya': np.round([0.06/0.29, 0.09/0.56, 0.14/1.7],3)*u.dex(u.erg/u.s),
    'EW_rest': [23, 71, 37]*u.angstrom,
    'e_EW_rest': [6, 18, 3]*u.angstrom,
    'logLLya': np.round(np.log10([0.2*10**(43), 0.4*10**(43), 1.2*10**(43)]),3)*u.dex(u.erg/u.s),
    'e_logLLya': np.round([0.1/0.2, 0.1/0.4, 0.1/1.2],3)*u.dex(u.erg/u.s),
    'HII_radii': [0.55, 0.69, 1.02]*pMpc,
    'SNR': [4.9, 6.0, 12.1],
}

Tilvi20 = Table(data_Tilvi20)
Tilvi20

ID,RA_J2000,DEC_J2000,NB,e_NB,F125W,e_F125W,F160W,e_F160W,z_spec,logfLya,e_logfLya,EW_rest,e_EW_rest,logLLya,e_logLLya,HII_radii,SNR
,,,mag,mag,mag,mag,mag,mag,,dex(erg / s),dex(erg / s),Angstrom,Angstrom,dex(erg / s),dex(erg / s),pMpc,
str5,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
z8_SM,14:20:35.694,+53:00:09.318,24.76,0.35,26.76,0.13,26.66,0.11,7.767,-17.538,0.207,23.0,6.0,42.301,0.5,0.55,4.9
z8_4,14:20:35.169,+52:59:40.613,23.85,0.15,27.25,0.21,27.1,0.16,7.748,-17.252,0.161,71.0,18.0,42.602,0.25,0.69,6.0
z8_5,14:20:34.872,+53:00:15.242,23.6,0.12,25.29,0.03,25.08,0.03,7.728,-16.77,0.082,37.0,3.0,43.079,0.083,1.02,12.1


## 2. Filter in redshift

Utilzaremos los datos de Tilvi20 para la realización de este cuaderno escogiendo únicamente aquellas galaxias con z > 6, puesto que estas son las únicas para las cuales el Universo todavía no estaba completamente reionizado y por tanto, estas esferas tenían un tamaño finito (no ocupaban todo el universo). 

In [3]:
Tilvi20_filter = Tilvi20[Tilvi20['z_spec'] > 6]
Tilvi20_filter

ID,RA_J2000,DEC_J2000,NB,e_NB,F125W,e_F125W,F160W,e_F160W,z_spec,logfLya,e_logfLya,EW_rest,e_EW_rest,logLLya,e_logLLya,HII_radii,SNR
,,,mag,mag,mag,mag,mag,mag,,dex(erg / s),dex(erg / s),Angstrom,Angstrom,dex(erg / s),dex(erg / s),pMpc,
str5,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
z8_SM,14:20:35.694,+53:00:09.318,24.76,0.35,26.76,0.13,26.66,0.11,7.767,-17.538,0.207,23.0,6.0,42.301,0.5,0.55,4.9
z8_4,14:20:35.169,+52:59:40.613,23.85,0.15,27.25,0.21,27.1,0.16,7.748,-17.252,0.161,71.0,18.0,42.602,0.25,0.69,6.0
z8_5,14:20:34.872,+53:00:15.242,23.6,0.12,25.29,0.03,25.08,0.03,7.728,-16.77,0.082,37.0,3.0,43.079,0.083,1.02,12.1


## 3. Strömgren Radius

Ya es el momento de estimar el radio de Strömgen, o mejor dicho, el límite superior al tamaño de estas burbujas para cada galaxia a z > 6. Este radio se define como:

$$
\begin{equation}
    R_s = \left(\frac{3 \dot{N}_{ion}}{4\pi C_H \langle n_H \rangle ^ 2 \alpha_B}\right)^{1/3}
\end{equation}
$$

donde $\dot{N}_{ion}$ se corresponde con el número de fotones del continuo ionizante, según las definciones en [Reionized-Bubble-published](https://watermark.silverchair.com/mnrasl_495_1_l17.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAA3UwggNxBgkqhkiG9w0BBwagggNiMIIDXgIBADCCA1cGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMesIRyAw1IMhjJgxLAgEQgIIDKNLnPEZWRv4H07eC47gxYs3YVUGv5PmSrfVEViCuE9OGjq7ro9MHhMyPmAakzFX36UUoujJ_iE9CnliM-I7rMh4z7mOeffZzdY6hfOyq00dea8-Lq1pWKHV4W24VMD60GNXdkNjd2aavHQncseS1xETCLGjWV75goKbuh_yg9nXTHI6v6S2MA0fKTlzk56zTvu3069rRq-WpUdLuyv0U6NSQNNtqfWMR468NbLLTfIHve9wfcSxGB-iHQiIJD0nQVAkDzO-LT1nxBzsiES0EvVeGqMNtyfLWRrMX8NxIJaaGEljDw5QQtk3ogx7j5QCeYpUBYY7__mlZBc8-pdDzOQ5ALXsasW69Qkf1sSmFzIvTPaMEK9YUN8axtA9iJViI5U33clBDs8xa0nJi5GA1IsnB9Elr1WP06eTj6v-HQeNqO5w4vP9vKIYTsDRvFzuu6U-Ok0Kw0aH2ybnJAVYQ8qfFRo29CcuPwsu8UVwCpISBnRl-269GGOHYzPoEzJrp_dWBnvXTYh6TTl0178RJhmDySOlH5Ie4tMfe-dp4cdQnhl0z-x4MQRIG_SDb3Xo-crH844EffwRQ5EnsktfdoMwXXvndvakMYFzSCSYEnJU1ZhMCUPH3IeyMQS1IXNM05Qm8QGW1OlUrqIvLDMjlCZeWptQ6X7mX5lX9jph4M9eKAelJw-dZjqlhm6NHjGS7FjhfsczkoehMAi87W5bm6FPbQlTpzEdDoNfMcc3GPH1SEbdltmN8j6TrfTA5pk2JFqntSzJJC99DHJzDeSWGumc1zg4FrP7vkUC4yAGgoMsSac6IL_SpwXEKc9sKB9ezu5PVy4NrWL7sc-xSmau0sVn8g_wP25TePV25N4Z8GxAE1cV_PPHKOFJU21pomgFaZdxyPxENLoyx9v64JuoXhi48PWuV9V1JPAXatkS2q3VTz5hRGiUD9nxqy7DbU6jVmZwqErVDNamfyN_0H7VxMEOm5sa65jZyHaGtyknwQq6fktLmskgK_kmotgnM2rZWexXFBGzK5USVEwHe-iA7Ri0pKvudfX8MtFVjs8HZGOqBvO118nGdD3Q). Esto es,

$$
\begin{equation}
    \dot{N}_{ion} = \frac{Q_{ion}}{1-f_{esc,LyC}} \cdot f_{esc,LyC}
\end{equation}
$$

donde $Q_{ion}$ se corresponde con los valores de $10^{logN_{ion}}$ de la tabla de Kerutt22 $\textbf{(cuidado con la notación)}$. Se asume $f_{esc,LyC} = 0.2$, valor estimador por diversos autores. 

Para realizar este cálculo, vamos a crear una función que lo calcule directamente a partir de la anchura equivalente (en angstroms), el logaritmo en base 10 de la luminosidad observada de la línea de lymann alpha (en dex(erg/s) y el redsfhit (adimensional). 

In [4]:
# Strömgen radius calculation using equivalent width (EW), log10(observed luminosity of Lymann alpha) and redshift
# Units: EW in angstroms; logLLya in dex(erg/s); z (adimensional)
def R_s(EW, e_EW, logLLya, e_logLLya, redshift):

    # recombination coefficinet for T_e = 10⁴K
    alphaB = 8.8 * 10**(-87)
    
    # C_H(z) parameter
    def C_H(redshift):
        z_values = [6 ,7 , 8]
        C_H_values = [2.37, 2.28, 2.19]
        C_H = np.interp(redshift, z_values,C_H_values) 
        return C_H

    # Hydrogen density
    def n_H (redshift):
        n_H = 5.878 * 10**66 * (1+redshift)**3
        return n_H

    # Escape fraction of Lymann alpha
    f_esc_Lya = []

    for i in range(len(EW)):
        if 0 <= EW[i] <= 200:
            f_esc_Lya.append(0.0048*EW[i])
        else:
            f_esc_Lya.append(1)

    e_f_esc_Lya = 0.0048 * e_EW

    # log10(intrisic luminosity of Lymann alpha)
    logLLya_int = np.log10(10**logLLya/f_esc_Lya)
    e_logLLya_int = np.sqrt( ((e_logLLya*np.log(10)*10**logLLya)/f_esc_Lya)**2 + (e_f_esc_Lya/(10**logLLya))** 2); e_logLLya_int = e_logLLya_int / (10**logLLya * np.log(10))

    # Effective number of ionizinz continuum photones per second
    logQ_ion = np.log10(10**logLLya_int / (1.19*10**(-11)))
    e_logQ_ion = (1/np.log(10)) * (1/10**logQ_ion) * ((e_logLLya_int*np.log(10)*10**logLLya_int)/(1.19*10**(-11)))

    # Nuber of ionizing continuum photones that participate in reionizing the IGM
    f_esc_LyC = 0.2 # assumed f_esc_LyC
    N_ion = (10**logQ_ion) / (1 - f_esc_LyC) * f_esc_LyC
    e_logN_ion = (1/np.log(10)) * (1/N_ion) * (e_logQ_ion*np.log(10)*10**logQ_ion) * f_esc_LyC / (1 - f_esc_LyC)

    # Strömgen radius (proper)
    R_s = (3 * N_ion / (4 * np.pi * C_H(redshift) * n_H(redshift)**2 * alphaB))**(1/3)
    e_R_s = 1/3 * (3 * (e_logN_ion * np.log(10) * N_ion) / (4 * np.pi * C_H(redshift) * n_H(redshift)**2 * alphaB)) * (3 * (N_ion) / (4 * np.pi * C_H(redshift) * n_H(redshift)**2 * alphaB))**(-2/3)

    # Strömgen radius (comovil)
    R_s_com = (1 + redshift) * R_s
    e_R_s_com = (1 + redshift) * e_R_s
    
    return np.round(R_s,3) * pMpc, np.round(R_s_com,3) * cMpc, np.round(f_esc_Lya,3), np.round(logLLya_int,3) * u.dex(u.erg / u.s), np.round(logQ_ion,3)  * u.dex(1 / u.s), np.round(EW,2)  * u.angstrom, np.round(np.log10(N_ion),3) * u.dex(1 / u.s),  np.round(e_f_esc_Lya,3), np.round(e_logLLya_int,3) * u.dex(u.erg / u.s), np.round(e_logN_ion,3) * u.dex(1 / u.s), np.round(e_R_s,3) * pMpc, np.round(e_R_s_com,3) * cMpc

In [5]:
Tilvi20_filter['R_s'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[0]
Tilvi20_filter['e_R_s'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[10]
Tilvi20_filter['R_s_com'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[1]
Tilvi20_filter['e_R_s_com'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[11]
Tilvi20_filter['f_esc_Lya'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[2]
Tilvi20_filter['e_f_esc_Lya'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[7]
Tilvi20_filter['logLLya_int'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[3]
Tilvi20_filter['e_logLLya_int'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[8]
#Tilvi20_filter['logQ_ion'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['z_spec'])[4]
Tilvi20_filter['logN_ion'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[6]
Tilvi20_filter['e_logN_ion'] = R_s(Tilvi20_filter['EW_rest'],Tilvi20_filter['e_EW_rest'],Tilvi20_filter['logLLya'],Tilvi20_filter['e_logLLya'],Tilvi20_filter['z_spec'])[9]
Tilvi20_filter

ID,RA_J2000,DEC_J2000,NB,e_NB,F125W,e_F125W,F160W,e_F160W,z_spec,logfLya,e_logfLya,EW_rest,e_EW_rest,logLLya,e_logLLya,HII_radii,SNR,R_s,e_R_s,R_s_com,e_R_s_com,f_esc_Lya,e_f_esc_Lya,logLLya_int,e_logLLya_int,logN_ion,e_logN_ion
,,,mag,mag,mag,mag,mag,mag,,dex(erg / s),dex(erg / s),Angstrom,Angstrom,dex(erg / s),dex(erg / s),pMpc,,pMpc,pMpc,cMpc,cMpc,,,dex(erg / s),dex(erg / s),dex(1 / s),dex(1 / s)
str5,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
z8_SM,14:20:35.694,+53:00:09.318,24.76,0.35,26.76,0.13,26.66,0.11,7.767,-17.538,0.207,23.0,6.0,42.301,0.5,0.55,4.9,0.668,2.321,5.854,20.348,0.11,0.029,43.258,4.529,53.58,4.529
z8_4,14:20:35.169,+52:59:40.613,23.85,0.15,27.25,0.21,27.1,0.16,7.748,-17.252,0.161,71.0,18.0,42.602,0.25,0.69,6.0,0.58,0.327,5.075,2.857,0.341,0.086,43.07,0.734,53.392,0.734
z8_5,14:20:34.872,+53:00:15.242,23.6,0.12,25.29,0.03,25.08,0.03,7.728,-16.77,0.082,37.0,3.0,43.079,0.083,1.02,12.1,1.044,0.374,9.112,3.269,0.178,0.014,43.83,0.467,54.152,0.467


Los radios de Strömghen calculados a partir de Sobral (Tilvi20_filter[R_s]) son parecidos a las estimaciones realizadas a partir de la relación de Yajima (Tilvi20_filter[HII_Radii]). No obstante, debemos de estimar los errores antes para corroborar esta relación. 

Nótese que la notación de R_s y R_s_com se refieren a los radios de Strömgren calculados a partir de al relación de Sobral mientras que el denominado como HII_Radii es el radio de Strömgren calculado a partir de la relación de Yajima.


### 4.1. Errors in Strömgren Radius

Es cierto que hemos calculado los errores del radio de Strömgren para los datos pero no obstante, vamos a hacerlo usando el apquete de `uncertainties` para corroborar que nos sale lo mismo. 

In [6]:
# Recombination coefficinet for T_e = 10⁴K
alphaB = 8.8 * 10**(-87)

# C_H(z) parameter
def C_H(redshift):
    z_values = [6 ,7 , 8]
    C_H_values = [2.37, 2.28, 2.19]
    C_H = np.interp(redshift, z_values,C_H_values) 
    return C_H

# Hydrogen density
def n_H (redshift):
    n_H = 5.878 * 10**66 * (1+redshift)**3
    return n_H

# Escape fraction
f_esc_LyC = 0.2

# Errors in Strömgen Radius
N_unc_e = unumpy.uarray(10**Tilvi20_filter['logN_ion'].value, (10**Tilvi20_filter['logN_ion'].value*Tilvi20_filter['e_logN_ion'].value) / (1 - f_esc_LyC) * f_esc_LyC)
C_H_unc = unumpy.uarray(C_H(Tilvi20_filter['z_spec']),0)
n_H_unc = unumpy.uarray(n_H(Tilvi20_filter['z_spec']),0)

e_R_s_values =  (3 * N_unc_e / (4 * np.pi * C_H_unc * n_H_unc**2 * alphaB))**(1/3)
e_R_s = []

for i in range(len(e_R_s_values)):
    e_R_s.append(e_R_s_values[i].std_dev)

Tilvi20_filter['e_R_s'] = np.round(e_R_s,3) * pMpc
Tilvi20_filter['e_R_s_com'] = np.round(e_R_s*(1+Tilvi20_filter['z_spec']),3) * cMpc
Tilvi20_filter

ID,RA_J2000,DEC_J2000,NB,e_NB,F125W,e_F125W,F160W,e_F160W,z_spec,logfLya,e_logfLya,EW_rest,e_EW_rest,logLLya,e_logLLya,HII_radii,SNR,R_s,e_R_s,R_s_com,e_R_s_com,f_esc_Lya,e_f_esc_Lya,logLLya_int,e_logLLya_int,logN_ion,e_logN_ion
,,,mag,mag,mag,mag,mag,mag,,dex(erg / s),dex(erg / s),Angstrom,Angstrom,dex(erg / s),dex(erg / s),pMpc,,pMpc,pMpc,cMpc,cMpc,,,dex(erg / s),dex(erg / s),dex(1 / s),dex(1 / s)
str5,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
z8_SM,14:20:35.694,+53:00:09.318,24.76,0.35,26.76,0.13,26.66,0.11,7.767,-17.538,0.207,23.0,6.0,42.301,0.5,0.55,4.9,0.668,0.252,5.854,2.209,0.11,0.029,43.258,4.529,53.58,4.529
z8_4,14:20:35.169,+52:59:40.613,23.85,0.15,27.25,0.21,27.1,0.16,7.748,-17.252,0.161,71.0,18.0,42.602,0.25,0.69,6.0,0.58,0.035,5.075,0.31,0.341,0.086,43.07,0.734,53.392,0.734
z8_5,14:20:34.872,+53:00:15.242,23.6,0.12,25.29,0.03,25.08,0.03,7.728,-16.77,0.082,37.0,3.0,43.079,0.083,1.02,12.1,1.044,0.041,9.112,0.355,0.178,0.014,43.83,0.467,54.152,0.467


## Appendix A: Saving data

Si se ejecuta el siguiente código, guardaremos y sobreescribiremo los datos. Nótese que lo guardamos en formato ecsv (formato profesional para astrofísica)

In [7]:
ascii.write(Tilvi20_filter, 'Tilvi20_filter_v1.dat', format='ecsv', overwrite=True)  


In [10]:
# To LaTeX
data_table = ascii.read('Tilvi20_filter_v1.dat', format='ecsv')
df = data_table.to_pandas()
path = '/home/juanan/Escritorio/Juanan/Estudios/Master_en_Astrofisica/Practicas_ESAC/MyJob/latex_tables/Tilvi20.tex'
cols = ['ID', 'z_spec', 'EW_rest', 'logLLya', 'f_esc_Lya', 'R_s', 'HII_radii']
df.to_latex(buf=path ,columns= cols, float_format='%.2f',header=True, index=False, position= 'H', label = 'Tilvi20', caption = 'Tilvi20')